# kaggle dataset link-https://www.kaggle.com/datasets/karkavelrajaj/amazon-sales-dataset?resource=download

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel,sigmoid_kernel
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings as wr
wr.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns',None)
dataset = pd.read_csv('amazon.csv')
dataset.reset_index(drop=True,inplace=True)


In [3]:
dataset.head()

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   product_id           1465 non-null   object
 1   product_name         1465 non-null   object
 2   category             1465 non-null   object
 3   discounted_price     1465 non-null   object
 4   actual_price         1465 non-null   object
 5   discount_percentage  1465 non-null   object
 6   rating               1465 non-null   object
 7   rating_count         1463 non-null   object
 8   about_product        1465 non-null   object
 9   user_id              1465 non-null   object
 10  user_name            1465 non-null   object
 11  review_id            1465 non-null   object
 12  review_title         1465 non-null   object
 13  review_content       1465 non-null   object
 14  img_link             1465 non-null   object
 15  product_link         1465 non-null   object
dtypes: obj

In [5]:
dataset.describe(include='object')

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
count,1465,1465,1465,1465,1465,1465,1465,1463,1465,1465,1465,1465,1465,1465,1465,1465
unique,1351,1337,211,550,449,92,28,1143,1293,1194,1194,1194,1194,1212,1412,1465
top,B08WRWPM22,"Fire-Boltt Ninja Call Pro Plus 1.83"" Smart Wat...",Computers&Accessories|Accessories&Peripherals|...,₹199,₹999,50%,4.1,"9,378",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AHIKJUDTVJ4T6DV6IUGFYZ5LXMPA,AE55KTFVNXYFD5FPY...","$@|\|TO$|-|,Sethu madhav,Akash Thakur,Burger P...","R3F4T5TRYPTMIG,R3DQIEC603E7AY,R1O4Z15FD40PV5,R...","Worked on iPhone 7 and didn’t work on XR,Good ...","I am not big on camera usage, personally. I wa...",https://m.media-amazon.com/images/I/413sCRKobN...,https://www.amazon.in/Borosil-Jumbo-1000-Watt-...
freq,3,5,233,53,120,56,244,9,6,10,10,10,10,8,3,1


Preprocessing

In [6]:
dataset.rating = pd.to_numeric(dataset.rating,errors='coerce',downcast='float')#changing to float value
dataset.rating_count = dataset.rating_count.str.replace(',','') # routing_count column contains delimiter(,) so replacing with empty string
dataset.rating_count = pd.to_numeric(dataset.rating_count,errors='coerce',downcast='float')#changing to float value

In [7]:
# Checking NUll values
dataset.isnull().sum()

product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 1
rating_count           2
about_product          0
user_id                0
user_name              0
review_id              0
review_title           0
review_content         0
img_link               0
product_link           0
dtype: int64

In [8]:
dataset.dropna(inplace=True)#dropping null values
dataset.isnull().sum()

product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0
rating_count           0
about_product          0
user_id                0
user_name              0
review_id              0
review_title           0
review_content         0
img_link               0
product_link           0
dtype: int64

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1462 entries, 0 to 1464
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   product_id           1462 non-null   object 
 1   product_name         1462 non-null   object 
 2   category             1462 non-null   object 
 3   discounted_price     1462 non-null   object 
 4   actual_price         1462 non-null   object 
 5   discount_percentage  1462 non-null   object 
 6   rating               1462 non-null   float32
 7   rating_count         1462 non-null   float32
 8   about_product        1462 non-null   object 
 9   user_id              1462 non-null   object 
 10  user_name            1462 non-null   object 
 11  review_id            1462 non-null   object 
 12  review_title         1462 non-null   object 
 13  review_content       1462 non-null   object 
 14  img_link             1462 non-null   object 
 15  product_link         1462 non-null   object

###Recomendation system using content based filtering###


In [10]:
dataset.product_name.values


array(['Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)',
       'Ambrane Unbreakable 60W / 3A Fast Charging 1.5m Braided Type C Cable for Smartphones, Tablets, Laptops & other Type C devices, PD Technology, 480Mbps Data Sync, Quick Charge 3.0 (RCT15A, Black)',
       'Sounce Fast Phone Charging Cable & Data Sync USB Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini & iOS Devices',
       ...,
       'Bajaj Majesty RX10 2000 Watts Heat Convector Room Heater (White, ISI Approved)',
       'Havells Ventil Air DSP 230mm Exhaust Fan (Pista Green)',
       'Borosil Jumbo 1000-Watt Grill Sandwich Maker (Black)'],
      dtype=object)

In [11]:
dataset.category.values[1:10]

array(['Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables',
       'Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables',
       'Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables',
       'Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables',
       'Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables',
       'Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables',
       'Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables',
       'Computers&Accessories|NetworkingDevices|NetworkAdapters|WirelessUSBAdapters',
       'Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables'],
      dtype=object)

In [12]:
dataset["product_category"] = dataset.category.apply(lambda x : x.split('|')[-1])

In [13]:
dataset.head()

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link,product_category
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,24269.0,High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...,USBCables
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,43994.0,"Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...,USBCables
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",90%,3.9,7928.0,【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...,USBCables
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,53%,4.2,94363.0,The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...,USBCables
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,61%,4.2,16905.0,[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...,USBCables


In [14]:
data = dataset.loc[:,['product_name','product_category']]
list(data.product_category.unique())

['USBCables',
 'WirelessUSBAdapters',
 'HDMICables',
 'SmartTelevisions',
 'RemoteControls',
 'StandardTelevisions',
 'TVWall&CeilingMounts',
 'RCACables',
 'Mounts',
 'OpticalCables',
 'Projectors',
 'Adapters',
 'SatelliteReceivers',
 'DVICables',
 'SpeakerCables',
 'StreamingClients',
 'AVReceivers&Amplifiers',
 'TowerSpeakers',
 '3DGlasses',
 'SmartWatches',
 'PowerBanks',
 'Smartphones',
 'MicroSD',
 'BasicMobiles',
 'In-Ear',
 'AutomobileChargers',
 'Cradles',
 'WallChargers',
 'OTGAdapters',
 'Tripods',
 'SelfieSticks',
 'Stands',
 'CableConnectionProtectors',
 'Décor',
 'ScreenProtectors',
 'StylusPens',
 'Bedstand&DeskMounts',
 'BasicCases',
 'HandlebarMounts',
 'On-Ear',
 'CameraPrivacyCovers',
 'PhoneCharms',
 'Shower&WallMounts',
 'PenDrives',
 'Mice',
 'GraphicTablets',
 'Lapdesks',
 'NotebookComputerStands',
 'Keyboards',
 'Condenser',
 'DisposableBatteries',
 'GelInkRollerballPens',
 'Tape',
 'Keyboard&MouseSets',
 'ExternalHardDisks',
 'VideoCameras',
 'Tabletop&TravelT

In [15]:
data.head()

,product_name,product_category
0,Wayona Nylon Braided USB to Lightning Fast Cha...,USBCables
1,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,USBCables
2,Sounce Fast Phone Charging Cable & Data Sync U...,USBCables
3,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,USBCables
4,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,USBCables


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1462 entries, 0 to 1464
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_name      1462 non-null   object
 1   product_category  1462 non-null   object
dtypes: object(2)
memory usage: 34.3+ KB


In [17]:
# changing to lower case
data.product_name=data.product_name.str.lower()
data.product_category = data.product_category.str.lower()

# now we need to convert the product_name to vector after preprocessing

In [19]:
tfidf = TfidfVectorizer(ngram_range=(1,3),stop_words='english')

In [20]:
tfidf_matrix=tfidf.fit_transform(data.product_name)
len(tfidf.vocabulary_)

33387

In [21]:
cosine_sim_scores = cosine_similarity(tfidf_matrix,tfidf_matrix)

In [22]:
data[data.product_category == 'wetgrinders']

,product_name,product_category
1205,"butterfly smart wet grinder, 2l (white) with c...",wetgrinders
1345,"prestige pwg 07 wet grinder, 2l (multicolor) w...",wetgrinders


In [23]:
def recommendation_system(category_name : str  ,score: np.array([]) = cosine_sim_scores,top_n : int = 3):
    
    #idx = data.product_category.isin([category_name.lower()]).index[0]#storing the first index value of where it matches the category
    if category_name.lower() in data.product_category.str.lower().values :
        idx = data[data.product_category == category_name.lower()].index[0]
    else:
        raise 'your searched product is not their in our e-commerce data set'
    sim = sorted(list(enumerate(score[idx])),reverse=True,key = lambda x: x[1])
    sim_scores = sim[1:top_n+1]
    property_indices = [i[0] for i in sim_scores]
    recommendation_df = pd.DataFrame({
        'product_name': data['product_name'].iloc[property_indices],
        'similarity_scores':sim_scores[0][1]
                                    }
                                    )
    return recommendation_df.sort_values(by='similarity_scores').reset_index().loc[:,['product_name','similarity_scores']]


In [28]:
category_input = input("Enter the product category: ")
top_n_input = int(input("Enter the number of top recommendations: "))

Enter the product category:  Smartphones
Enter the number of top recommendations:  8


In [29]:
recommended_products=recommendation_system(category_input,top_n=top_n_input)

In [30]:
recommended_products

,product_name,similarity_scores
0,"oneplus nord 2t 5g (jade fog, 8gb ram, 128gb s...",0.540966
1,"oneplus nord 2t 5g (jade fog, 12gb ram, 256gb ...",0.540966
2,"oneplus 10t 5g (moonstone black, 8gb ram, 128g...",0.540966
3,"oneplus 10r 5g (forest green, 8gb ram, 128gb s...",0.540966
4,"iqoo neo 6 5g (dark nova, 8gb ram, 128gb stora...",0.540966
5,"samsung galaxy s20 fe 5g (cloud navy, 8gb ram,...",0.540966
6,"iqoo 9 se 5g (sunset sierra, 8gb ram, 128gb st...",0.540966
7,"iqoo z6 pro 5g by vivo (legion sky, 8gb ram, 1...",0.540966


In [31]:
for products in recommended_products.product_name.values:
    print(products)

oneplus nord 2t 5g (jade fog, 8gb ram, 128gb storage)
oneplus nord 2t 5g (jade fog, 12gb ram, 256gb storage)
oneplus 10t 5g (moonstone black, 8gb ram, 128gb storage)
oneplus 10r 5g (forest green, 8gb ram, 128gb storage, 80w supervooc)
iqoo neo 6 5g (dark nova, 8gb ram, 128gb storage) | snapdragon® 870 5g | 80w flashcharge
samsung galaxy s20 fe 5g (cloud navy, 8gb ram, 128gb storage) with no cost emi & additional exchange offers
iqoo 9 se 5g (sunset sierra, 8gb ram, 128gb storage) | qualcomm snapdragon 888 | 66w flash charge
iqoo z6 pro 5g by vivo (legion sky, 8gb ram, 128gb storage) | snapdragon 778g 5g | 66w flashcharge | 1300 nits peak brightness | hdr10+
